In [1]:
import sys
import os
from datetime import datetime
import copy
import numpy as np
import pandas as pd
from collections import Counter
from functools import reduce
import matplotlib.pyplot as plt
from scipy.optimize import minimize, LinearConstraint

sys.path.append(os.getcwd())

import ContractVolatile as trinity

#POOL setup, Mary deposit as the protocol, Another external LP as Others and Peter act as arbitrague and swap around.
# Create the Pool
Trinity = trinity.Pool()

# Create the Users
Peter = trinity.Users("Peter")  # Peter is the user
Mary = trinity.Users("Mary")  # Mary is the arbitrageur
Others = trinity.Users(
    "Others"
)  # This is an external system (for book keeping of arbitrageur)

"""
Block begin user inputs.
Change all inputs, including tokens, parameters setup here
"""

# Create all the tokens here
BTC = trinity.Tokens("BTC")  # *** To be modified ***
ETH = trinity.Tokens("ETH")  # *** To be modified ***
USDT = trinity.Tokens("USDT")  # *** To be modified ***

#Printing parameters
print_update = False
print_action = True
no_fee = False
trinity.no_fee = no_fee
trinity.print_action = print_action
trinity.print_update = print_update

# User wallet, BTC=30000, ETH=2000, Denote Peter to be arbitraguer swapper, Mary to be first depositor, Other to be an active LP
Peter.wallet = {"BTC": 100, "ETH": 3000, "USDT": 6000000}  # *** To be modified ***
Mary.wallet = {"BTC": 100, "ETH": 3000, "USDT": 6000000}  # *** To be modified ***
Others.wallet = {"BTC": 100, "ETH": 3000,"USDT": 6000000,}  # *** To be modified ***

Peter.lp_token = {"BTC": 0, "ETH": 0, "USDT": 0}
Mary.lp_token = {"BTC": 0, "ETH": 0, "USDT": 0}
Others.lp_token = {"BTC": 0, "ETH": 0, "USDT": 0}

"""
Block begin initialization after user inputs.
"""
Tokens = trinity.Tokens.get_instances()
m = len(Tokens)

# Setting Initial token price
price_list = ['price_scale', 'price_scale_new', 'price_oracle', 'price_market', 'price_last']
price_list_records = ['price_scale_records', 'price_scale_new_records','price_oracle_records', 'price_last_records']

for price_index in price_list:
    setattr(BTC, price_index, 30000)
    setattr(ETH, price_index, 2000)

for price_index_records in price_list_records:
    setattr(BTC, price_index_records, [30000])
    setattr(ETH, price_index_records, [2000])


#Recording initial state 
Users = trinity.Users.get_instances()
init_Users_state = {
    key: Peter.wallet[key] + Mary.wallet[key] + Others.wallet[key]
    for key in Peter.wallet
}
init_Users_lp_state = {
    key: Peter.lp_token[key] + Mary.lp_token[key] + Others.lp_token[key]
    for key in Peter.wallet
}

Mary_init = copy.deepcopy(Mary.wallet)
init_Pool_state = {"BTC": BTC.cash, "ETH": ETH.cash, "USDT": USDT.cash}
init_Pool_lp_token = {"BTC": BTC.lp_token, "ETH": ETH.lp_token, "USDT": USDT.lp_token}


In [2]:
#Determine the state by deposit 
# Initial Deposit
Mary.deposit(Trinity, BTC, 20, ETH, USDT)  # *** To be modified ***
Mary.deposit(Trinity, ETH, 300, BTC, USDT)  # *** To be modified ***
Mary.deposit(Trinity, USDT, 600000, BTC, ETH) # *** To be modified ***

# Users can also arbitrarily change the state:
# x=0
# BTC.cash  = x # To be modified
# ETH.cash  = x # To be modified
# USDT.cash = x # To be modified
# BTC.liability = x # To be modified
# ETH.liability = x # To be modified
# USDT.liability = x # To be modified


DEPOSIT: Mary deposited 20 BTC and gain reward of 0 (0.0%) adding total 20 BTC liability (20 LP) into the Pool
DEPOSIT: Mary deposited 300 ETH and gain reward of 0 (0.0%) adding total 300 ETH liability (300 LP) into the Pool
DEPOSIT: Mary deposited 600000 USDT and gain reward of 0 (0.0%) adding total 600000 USDT liability (600000 LP) into the Pool


In [3]:
print("---------- The initial state is --------------")
print({"Asset: BTC": BTC.cash, "ETH": ETH.cash, "USDT": USDT.cash})
print({"Liability: BTC": BTC.liability, "ETH": ETH.liability, "USDT": USDT.liability})
print({"Coverage ratio: BTC": BTC.cov_ratio(), "ETH": ETH.cov_ratio(), "USDT": USDT.cov_ratio()})
print({"hair cut: BTC": BTC.hf_lp_records, "ETH": ETH.hf_lp_records, "USDT": USDT.hf_lp_records})
print(f"Prices (oracle): BTC: {BTC.price_last} ({BTC.price_oracle}), ETH: {ETH.price_last} ({ETH.price_oracle}), USDT: {USDT.price_last} ({USDT.price_oracle}),")

print("------------------------------------------")
print("Mary Wallet is {0}".format(Mary.wallet))
print("Mary LP wallet is {0}".format(Mary.lp_token))




---------- The initial state is --------------
{'Asset: BTC': 20, 'ETH': 300, 'USDT': 600000}
{'Liability: BTC': 20, 'ETH': 300, 'USDT': 600000}
{'Coverage ratio: BTC': 1.0, 'ETH': 1.0, 'USDT': 1.0}
{'hair cut: BTC': 0, 'ETH': 0, 'USDT': 0}
Prices (oracle): BTC: 30000 (30000), ETH: 2000 (2000), USDT: 1 (1),
------------------------------------------
Mary Wallet is {'BTC': 80, 'ETH': 2700, 'USDT': 5400000}
Mary LP wallet is {'BTC': 20, 'ETH': 300, 'USDT': 600000}


In [4]:
Peter.swap_volatile_repeg(Trinity, BTC, 1, USDT, ETH)

print("---------- The state is --------------")
print({"Asset: BTC": BTC.cash, "ETH": ETH.cash, "USDT": USDT.cash})
print({"Liability: BTC": BTC.liability, "ETH": ETH.liability, "USDT": USDT.liability})
print({"Coverage ratio: BTC": BTC.cov_ratio(), "ETH": ETH.cov_ratio(), "USDT": USDT.cov_ratio()})
print({"hair cut: BTC": BTC.hf_lp_records, "ETH": ETH.hf_lp_records, "USDT": USDT.hf_lp_records})
print(f"Prices (oracle): BTC: {BTC.price_last} ({BTC.price_oracle}), ETH: {ETH.price_last} ({ETH.price_oracle}), USDT: {USDT.price_last} ({USDT.price_oracle}),")




SWAP: Peter swapped 1 BTC to 28568.428288487856 USDT. Fees charged: 9.830689792847465
The to_token USDT has 0.0 haircut with its price last 1 and price scale 1 
---------- The state is --------------
{'Asset: BTC': 21, 'ETH': 300, 'USDT': 571431.5717115122}
{'Liability: BTC': 20, 'ETH': 300, 'USDT': 600009.8423424903}
{'Coverage ratio: BTC': 1.05, 'ETH': 1.0, 'USDT': 0.9523703302609069}
{'hair cut: BTC': 0, 'ETH': 0, 'USDT': 9.842342490358561}
Prices (oracle): BTC: 27223.903073439516 (29999.999358585486), ETH: 1902.9410671620697 (1999.9999775746273), USDT: 1 (1),


In [5]:
Peter.swap_volatile_repeg(Trinity, USDT, 2000, ETH, BTC)

print("---------- The state is --------------")
print({"Asset: BTC": BTC.cash, "ETH": ETH.cash, "USDT": USDT.cash})
print({"Liability: BTC": BTC.liability, "ETH": ETH.liability, "USDT": USDT.liability})
print({"Coverage ratio: BTC": BTC.cov_ratio(), "ETH": ETH.cov_ratio(), "USDT": USDT.cov_ratio()})
print({"hair cut: BTC": BTC.hf_lp_records, "ETH": ETH.hf_lp_records, "USDT": USDT.hf_lp_records})
print(f"Prices (oracle): BTC: {BTC.price_last} ({BTC.price_oracle}), ETH: {ETH.price_last} ({ETH.price_oracle}), USDT: {USDT.price_last} ({USDT.price_oracle}),")

SWAP: Peter swapped 2000 USDT to 1.0469044260345146 ETH. Fees charged: 0.00036212320635917855
The to_token ETH has 0.00018106160317958928 haircut with its price last 1902.9410671620697 and price scale 2000 
******************************* Repegging Triggered *******************************
---------- The state is --------------
{'Asset: BTC': 21, 'ETH': 298.9529145123623, 'USDT': 573431.5717115122}
{'Liability: BTC': 20, 'ETH': 300.00018106270477, 'USDT': 600009.8423424903}
{'Coverage ratio: BTC': 1.05, 'ETH': 0.9965091136057563, 'USDT': 0.9557036089154566}
{'hair cut: BTC': 0, 'ETH': 0.0001810627047620294, 'USDT': 9.842342490358561}
Prices (oracle): BTC: 27323.195839907658 (29999.99874011262), ETH: 1916.5501707154006 (1999.9999582936302), USDT: 1 (1),


In [6]:
Peter.swap_volatile_repeg(Trinity, ETH, 2, BTC, USDT)

print("---------- The state is --------------")
print({"Asset: BTC": BTC.cash, "ETH": ETH.cash, "USDT": USDT.cash})
print({"Liability: BTC": BTC.liability, "ETH": ETH.liability, "USDT": USDT.liability})
print({"Coverage ratio: BTC": BTC.cov_ratio(), "ETH": ETH.cov_ratio(), "USDT": USDT.cov_ratio()})
print({"hair cut: BTC": BTC.hf_lp_records, "ETH": ETH.hf_lp_records, "USDT": USDT.hf_lp_records})
print(f"Prices (oracle): BTC: {BTC.price_last} ({BTC.price_oracle}), ETH: {ETH.price_last} ({ETH.price_oracle}), USDT: {USDT.price_last} ({USDT.price_oracle}),")

SWAP: Peter swapped 2 ETH to 0.1393354298135527 BTC. Fees charged: 4.593956336765788e-05
The to_token BTC has 2.296978168382894e-05 haircut with its price last 27323.195839907658 and price scale 30000 
******************************* Repegging Triggered *******************************
---------- The state is --------------
{'Asset: BTC': 20.860641600404765, 'ETH': 300.9529145123623, 'USDT': 573431.5717115122}
{'Liability: BTC': 20.0000229900636, 'ETH': 300.00018106270477, 'USDT': 600009.8423424903}
{'Coverage ratio: BTC': 1.0430308810529236, 'ETH': 1.0031757762488098, 'USDT': 0.9557036089154566}
{'hair cut: BTC': 2.2990063599201052e-05, 'ETH': 0.0001810627047620294, 'USDT': 9.842342490358561}
Prices (oracle): BTC: 27496.520518040532 (29999.998161686395), ETH: 1903.8753453997228 (1999.9999360841311), USDT: 1 (1),


In [7]:
print("------------- Leakage Test -------------")
# Leakage Test
for token in Tokens:
    user_cash = 0
    for user in Users:
        user_cash += user.wallet[token.name]
    token_end_balance = token.cash + token.hf_sys_records + user_cash + token.hf_repeg 
    print("{0} token cash is {1}".format(token.name, token.cash))
    leakage = init_Users_state[token.name] - token_end_balance
    print("{0} leakage is {1}".format(token.name, leakage))


------------- Leakage Test -------------
BTC token cash is 20.860641600404765
BTC leakage is -5.684341886080802e-14
ETH token cash is 300.9529145123623
ETH leakage is 0.0
USDT token cash is 573431.5717115122
USDT leakage is 0.0
